In [1]:
# Install required packages in the current Jupyter kernel
# Uncomment the following lines if you need to install these libraries
# If you run into permission issues, try with the --user option
#import sys
#!pip install -q rdflib networkx matplotlib
#!{sys.executable} -m pip install rdflib networkx matplotlib pandas stringdb --user


In [2]:
%load_ext autoreload
%autoreload 2
import sampler
import grabhogs_sparql
import map2string_fast
import rdflib
import SPARQLWrapper
import colour
import itertools
import networkx as nx
from matplotlib import pyplot as plt

In [3]:
datapath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/402676.protein.links.rdf.v11.5.txt.gz'
# RDF graph loading
rg = sampler.load_graph(datapath)

datapath2 = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.info.rdf.v11.5/402676.protein.info.rdf.v11.5.txt.gz'
# RDF graph loading
rg_info  = sampler.load_graph(datapath2)

rdflib Graph loaded successfully with 2620310 triples
rdflib Graph loaded successfully with 14559 triples


In [4]:
subjs = rg.subjects( unique = True)
seed = next(subjs)
print(seed)

https://string-db.org/network/402676.B6JZQ9


In [32]:
subg = sampler.sample( rg = rg , seed = seed,  layer_limit= 2 , sample_run = 10 )
print(set([p for p in subg.predicates()]))
print("rdflib Graph sampled successfully with {} triples".format(len(subg)))

166 654
{rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/any-confidence')}
rdflib Graph sampled successfully with 590 triples


In [42]:
subg = sampler.add_xrefs( rg_info , subg )
print("rdflib Graph annotated successfully with {} triples".format(len(subg)))
print(set([p for p in subg.predicates()]))
cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot")
i = 0 
for s,p,o in subg.triples((None, cross_ref, None)):
    print(s,p,o)
    i+= 1
    if i > 10:
        break

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
190
95
rdflib Graph annotated successfully with 685 triples
{rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/any-confidence'), rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot'), rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff')}
https://string-db.org/network/402676.B6JW12 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JW12
https://string-db.org/network/402676.B6JWL7 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JWL7
https://string-db.org/network/402676.B6JUY5 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JUY5
https://string-db.org/network/402676.B6JXF5 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JXF5
https://string-db.org/network/402676.B6JVD7 http://purl.org/lscr#x

In [48]:
proteins_by_species , results_subj_para , results_subj_ortho  = grabhogs_sparql.grab_hogs( subg , cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot") )

95
Total time for 190 SPARQL queries: 52.799238204956055 seconds (multiple batch calls in: 0 cases)
Num errors: 0


In [ ]:
orthograph =  grabhogs_sparql.grab_hogs_graph( subg , cross_ref , sparql_endpoint= None , USE_CASE = 1 , verbose = True , cross_ref_prop = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot"))

1
> /work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/RDF_Graphnet/rdf2network/grabhogs_sparql.py(90)query_SPARQL_with_limit_offset()
     88     import pdb ; pdb.set_trace()
     89 
---> 90     if(len(results_OMA["results"]["bindings"]) == LIMIT):
     91         # Request next set of results until end, append to previous
     92         res = results_OMA["results"]["bindings"]



ipdb>  print(result)


*** NameError: name 'result' is not defined


ipdb>  print(results)


*** NameError: name 'results' is not defined


ipdb>  next


TypeError: 'Document' object is not subscriptable
> /work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/RDF_Graphnet/rdf2network/grabhogs_sparql.py(90)query_SPARQL_with_limit_offset()
     88     import pdb ; pdb.set_trace()
     89 
---> 90     if(len(results_OMA["results"]["bindings"]) == LIMIT):
     91         # Request next set of results until end, append to previous
     92         res = results_OMA["results"]["bindings"]



ipdb>  print(results_OMA)


In [ ]:
print("rdflib Graph annotated successfully with {} triples".format(len(orthograph)))

In [ ]:
print(results_subj_para.head())
print(results_subj_ortho.head())

In [ ]:
server = 'dna079'
results_string_per_protein = map2string_fast.mapall(proteins_by_species , serverurl= "http://"+server+":3030/string_fuseki/sparql" )

In [ ]:

G = rdflib.extras.external_graph_libs.rdflib_to_networkx_multidigraph( subg , lambda s, p, o: {'data':{'key':p  , 'weight':1} } )


In [ ]:
#sample response
import time
start_time = time.time()
results_obj_para = grabhogs_sparql.query_orthologs("<http://purl.uniprot.org/uniprot/B6JXK8>", SPARQLWrapper.SPARQLWrapper("https://sparql.omabrowser.org/sparql/"))

end_time = time.time()
print("Total time : {} seconds".format(end_time - start_time))

In [ ]:
# example usage
uniprot_id = "http://purl.uniprot.org/uniprot/A0A3B5R6M3" 
serverurl = '
results =  map2string_fast.map2string_SPARQL(uniprot_id , serverurl = )
print(results)

In [ ]:

preds = [ p for p in rg.predicates( unique = True)]

red = colour.Color('red')
blue = colour.Color('blue')
c = [ c.hex_l for c in  list(red.range_to(blue, len(preds))) ]
colors = { p:c[i] for i,p in enumerate(preds)}
delta = 2/len(preds)
curve = { p:delta*i for i,p in enumerate(preds) }
style = itertools.cycle([ '-', '--' ])
line_style = { p:next(style) for p in preds }

In [ ]:
pos = nx.circular_layout( G )
f = plt.figure()
f.set_figwidth(15)
f.set_figheight(15)
plt.plot()
#plot the whole mess
ax = plt.gca()
for e in G.edges(data = True):
    ax.annotate("",
                xy=pos[e[0]], xycoords='data',
                xytext=pos[e[1]], textcoords='data',
                arrowprops=dict(arrowstyle="-", color=colors[e[2]['data']['key']],
                                shrinkA=5, shrinkB=5, lw = e[2]['data']['weight'], ls = line_style[e[2]['data']['key']],
                                patchA=None, patchB=None, alpha = e[2]['data']['weight'],
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.3*curve[e[2]['data']['key']])
                                ),
                                ),
                )

nx.draw_networkx_nodes(G, pos, node_color = 'b', node_size = 150, alpha = .5)
labels=nx.draw_networkx_labels(G , pos = pos , font_size= 15 , font_color='w')

plt.axis('off')
plt.show()


In [ ]:
#load bloom filters for string data
import pickle
import glob
def check_filters(val,filters = filters):
    for f in filters:
        if val in f[0][0]:
            return True
    return False
filters = './filters/bloomfinal_big*.pkl'
filters = glob.glob(filters)
filters = [ pickle.load(open(f, 'rb')) for f in filters ]
#check bloom filters for interactors in diff species
print(check_filters('crap'))
print(check_filters('23.BEL05_00185 _23.BEL05_00025 '.encode()))
print(check_filters('23.BEL05_00045 23.BEL05_02665'))

In [ ]:
#map string neighbours to OMA entries

#jump a few steps in HOGs

#fish for STRING interactions in other species


